In [35]:
import pandas as pd
from pathlib import Path
import re

In [36]:
path = Path("data/clean_data.xlsx")
df = pd.read_excel(path)
df.drop(["word_count", "need_correct"], axis=1, inplace=True)

df

,name,dates,teritory,sheet
0,Жорницька Феня Сигізмундівна,1924-1982,Територія,Ряд 1
1,Жорницький Яків Абрамович,1921-2006,Територія,Ряд 1
2,Шевченко Ганна Михайлівна,1912-1995,Територія,Ряд 1
3,Шевченко Филипп Иванович,1908-1979,Територія,Ряд 1
4,Балакирев Генадий Николаевич,1937-1999,Територія,Ряд 1
...,...,...,...,...
9343,Починка Григорій,1915-1983,біля туалету,Ряд 95
9344,невідомо,1915-1983,біля туалету,Ряд 95
9345,Шудра Омельян,1913-1983,біля туалету,Ряд 95
9346,Васько Марія,1940-2009,біля туалету,Ряд 95


In [37]:
word_pattern = r"[A-Za-zА-Яа-яІіЇїЄєҐґ’']+" #будь яка послідовність кирилиці або латиниці є словом

# колонка з кількістю слів
df["word_count"] = (
    df["name"]  
      .fillna("") #Заповнити null щоб не падало (просто страховка, вище замінив на J Dou])
      .str.replace(r"\s+", " ", regex=True).str.strip() # прибрати зайві пробіли
      .str.findall(word_pattern) # знайти матчі
      .str.len() # порахувати матчі
)

df_names = df[df['word_count']>1] #для статистики імен
# df_names = df #для статистики імен


In [38]:
# Проставити імя та призвіще
norm = (
    df_names["name"]
      .fillna("")
      .str.replace(r"\s+", " ", regex=True).str.strip()
)

df_names["second_name"]  = norm.str.extract(rf"^({word_pattern})", expand=False)
df_names["first_name"] = norm.str.extract(rf"^(?:{word_pattern})\s+({word_pattern})", expand=False)

df_names["first_name"] = df_names["first_name"].apply(
    lambda x: x[0].upper() + x[1:] if isinstance(x, str) and x else x
)


df_names.loc[df_names["first_name"] == "Мария", "first_name"] = "Марія"
df_names.loc[df_names["first_name"] == "Maria", "first_name"] = "Марія"
df_names.loc[df_names["first_name"] == "MaRya", "first_name"] = "Марія"
df_names.loc[df_names["first_name"] == "Марвя", "first_name"] = "Марія"
df_names.loc[df_names["first_name"] == "Маря", "first_name"] = "Марія"

df_names.loc[df_names["first_name"] == "Ганна", "first_name"] = "Анна"
df_names.loc[df_names["first_name"] == "анна", "first_name"] = "Анна"
df_names.loc[df_names["first_name"] == "Anna", "first_name"] = "Анна"

df_names.loc[df_names["first_name"] == "Aleksander", "first_name"] = "Олександр"
df_names.loc[df_names["first_name"] == "Александр", "first_name"] = "Олександр"
df_names.loc[df_names["first_name"] == "Алексан", "first_name"] = "Олександр"
df_names.loc[df_names["first_name"] == "Олексанр", "first_name"] = "Олександр"
df_names.loc[df_names["first_name"] == "Олександир", "first_name"] = "Олександр"
df_names.loc[df_names["first_name"] == "Олександ", "first_name"] = "Олександр"
df_names.loc[df_names["first_name"] == "Оександр", "first_name"] = "Олександр"
df_names.loc[df_names["first_name"] == "Олесандр", "first_name"] = "Олександр"

df_names.loc[df_names["first_name"] == "Олександрв", "first_name"] = "Олександра"
df_names.loc[df_names["first_name"] == "Олександа", "first_name"] = "Олександра"
df_names.loc[df_names["first_name"] == "Олексанра", "first_name"] = "Олександра"
df_names.loc[df_names["first_name"] == "Александра", "first_name"] = "Олександра"
df_names.loc[df_names["first_name"] == "Олескандра", "first_name"] = "Олександра"

df_names.loc[df_names["first_name"] == "Тарасик", "first_name"] = "Тарас"

df_names.loc[df_names["first_name"] == "София", "first_name"] = "Софія"

df_names.loc[df_names["first_name"] == "Юстина", "first_name"] = "Устина"
df_names.loc[df_names["first_name"] == "Юстинія", "first_name"] = "Устина"

df_names.loc[df_names["first_name"] == "Миколай", "first_name"] = "Микола"

df_names.loc[df_names["first_name"] == "Христинп", "first_name"] = "Христина"
df_names.loc[df_names["first_name"] == "Христя", "first_name"] = "Христина"
df_names.loc[df_names["first_name"] == "Хистина", "first_name"] = "Христина"


df_names.loc[df_names["first_name"] == "Натаоія", "first_name"] = "Наталія"
df_names.loc[df_names["first_name"] == "Наталья", "first_name"] = "Наталія"
df_names.loc[df_names["first_name"] == "Наталя", "first_name"] = "Наталія"
df_names.loc[df_names["first_name"] == "Наталия", "first_name"] = "Наталія"
df_names.loc[df_names["first_name"] == "Наташа", "first_name"] = "Наталія"
df_names.loc[df_names["first_name"] == "НАталія", "first_name"] = "Наталія"




# Вивести кількість імен
# with pd.option_context("display.max_rows", None):
#     print(df_names["first_name"].value_counts()[:100])


C:\Users\ПК\AppData\Local\Temp\ipykernel_4996\2860968816.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_names["second_name"]  = norm.str.extract(rf"^({word_pattern})", expand=False)
C:\Users\ПК\AppData\Local\Temp\ipykernel_4996\2860968816.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_names["first_name"] = norm.str.extract(rf"^(?:{word_pattern})\s+({word_pattern})", expand=False)
C:\Users\ПК\AppData\Local\Temp\ipykernel_4996\2860968816.py:11: SettingWithCopyWarning: 
A value is trying to be 

In [39]:
# топ найпопулярніших імен
with pd.option_context("display.max_rows", None):
    print(df_names["first_name"].value_counts())

first_name
Марія             510
Іван              362
Анна              317
Володимир         297
Микола            259
Олександр         186
Катерина          177
Михайло           175
Василь            171
Ольга             169
Петро             159
Надія             141
Степан            135
Олена             118
Віра              105
Йосип             102
Андрій            100
Софія              98
Анастасія          95
Федір              89
Григорій           87
Тетяна             87
Антоніна           83
Сергій             82
Павло              81
Олександра         79
Антон              77
Олексій            72
Ніна               69
Євгенія            69
Галина             68
Анатолій           67
Євдокія            64
Віктор             63
Наталія            61
Валентина          54
Дмитро             51
Лідія              50
Варвара            49
Людмила            44
Ірина              40
Любов              40
Юлія               39
Юрій               38
Ксенія             37

In [40]:
s = df["dates"].astype("string").str.replace(r"[–—−]", "-", regex=True)

# витягуємо ПЕРШИЙ рік до тире (народження) і ПЕРШИЙ рік після тире (смерті)
rng = s.str.extract(r'^\s*(?:.*?(\d{3,4})).*?-\s*(?:.*?(\d{3,4})).*$')

df["birth_year"] = rng[0]
df["death_year"] = rng[1]

# якщо в комірці був лише один рік без тире — це рік смерті
single = s.str.extract(r'^\s*(?:.*?(\d{3,4}))\s*$')
df["death_year"] = df["death_year"].fillna(single[0])

# привести до числового типу з підтримкою NaN
for col in ["birth_year", "death_year"]:
    df[col] = pd.to_numeric(df[col], errors="coerce").astype("Int64")

print(df)
df.to_excel("pre_data.xlsx")

                              name      dates      teritory   sheet  \
0     Жорницька Феня Сигізмундівна  1924-1982     Територія   Ряд 1   
1        Жорницький Яків Абрамович  1921-2006     Територія   Ряд 1   
2        Шевченко Ганна Михайлівна  1912-1995     Територія   Ряд 1   
3         Шевченко Филипп Иванович  1908-1979     Територія   Ряд 1   
4     Балакирев Генадий Николаевич  1937-1999     Територія   Ряд 1   
...                            ...        ...           ...     ...   
9343              Починка Григорій  1915-1983  біля туалету  Ряд 95   
9344                      невідомо  1915-1983  біля туалету  Ряд 95   
9345                 Шудра Омельян  1913-1983  біля туалету  Ряд 95   
9346                  Васько Марія  1940-2009  біля туалету  Ряд 95   
9347                 Васько Сергій  1962-1983  біля туалету  Ряд 95   

      word_count  birth_year  death_year  
0              3        1924        1982  
1              3        1921        2006  
2              3  

- найпопулярніші імена по даті народженню
- рік народження довгожителів
- рік в який народжувалися ті, хто живуть найменше
- найпопулярніші імена на кладовищі
- народження щороку по статях
- найвища смертність по роках

In [41]:
df['years'] = df['death_year'] - df['birth_year']
df = df[(df['years'] >= 100) & (df['years'] <= 135) &  (df["name"].str.strip().str.lower() != "невідома")]
df.to_excel("centenarians.xlsx", index=False)

In [42]:
df_names.head()

,name,dates,teritory,sheet,word_count,second_name,first_name
0,Жорницька Феня Сигізмундівна,1924-1982,Територія,Ряд 1,3,Жорницька,Феня
1,Жорницький Яків Абрамович,1921-2006,Територія,Ряд 1,3,Жорницький,Яків
2,Шевченко Ганна Михайлівна,1912-1995,Територія,Ряд 1,3,Шевченко,Анна
3,Шевченко Филипп Иванович,1908-1979,Територія,Ряд 1,3,Шевченко,Филипп
4,Балакирев Генадий Николаевич,1937-1999,Територія,Ряд 1,3,Балакирев,Генадий
